# Code computed Sentinel-2 NDWI on GEE and downloads

- Requires a GEE account
- Outputs will be saved to google drive and can be downloaded

In [ ]:
import ee
import geemap
from pyproj import Proj, transform

### Do this when you are using the notebook for the first time

- Uncomment the ee.Authenticate() line

In [ ]:
# Initialize the Earth Engine library. 
#ee.Authenticate()

In [ ]:
ee.Initialize()

## Define the bounding box and coordinate system for data

In [ ]:
# Define the UTM bounds
utm_zone = '22S'  # Example UTM zone
utm_bounds = {
    "x_min": 333550,
    "y_min": 9588140,
    "x_max": 441429,
    "y_max": 9800000
}
utm_proj = Proj(proj="utm", zone=22,south=True, ellps="WGS84")
wgs84_proj = Proj(proj="latlong", datum="WGS84")

# Convert UTM to geographic coordinates
lon_min, lat_min = transform(utm_proj, wgs84_proj, utm_bounds["x_min"], utm_bounds["y_min"])
lon_max, lat_max = transform(utm_proj, wgs84_proj, utm_bounds["x_max"], utm_bounds["y_max"])



aoi = ee.Geometry.Rectangle([lon_min, lat_min, lon_max, lat_max])

## Define the time range and cloud cover.


In [ ]:
# Fetch Sentinel-2 images
start_date = '2016-08-01'
end_date = '2016-12-31'

sentinel2_collection = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate(start_date, end_date) \
    .filterBounds(aoi) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))


## Compute NDWI

- currently it calculates an yearly median to compare with yearly SAR masks

In [ ]:
def compute_ndwi(image):
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    return image.addBands(ndwi)

# Map the NDWI function over the collection
sentinel2_ndwi = sentinel2_collection.map(compute_ndwi)

# Reduce the collection to a single image by taking the median
ndwi_image = sentinel2_ndwi.select('NDWI').median().clip(aoi)

## Display the Map

In [ ]:

# Create a map centered on the AOI
Map = geemap.Map(center=[(lat_min + lat_max) / 2, (lon_min + lon_max) / 2], zoom=10)

# Add the NDWI layer to the map
ndwi_params = {
    'min': -1,
    'max': 1,
    'palette': ['blue', 'white', 'green']
}
Map.addLayer(ndwi_image, ndwi_params, 'NDWI')

# Display the map
Map

## Export to Google drive.

- Input the image name in the description

In [ ]:
export_task = ee.batch.Export.image.toDrive(**{
    'image': ndwi_image,
    'description': 'NDWI_Map_Sentinel2_Xingu_2016',
    'folder': 'EarthEngineImages',
    'scale': 10,
    'region': aoi,
    'fileFormat': 'GeoTIFF',
    'maxPixels': 1e9
})

export_task.start()

import time

while export_task.active():
    print('Polling for task (id: {}).'.format(export_task.id))
    time.sleep(30)  # Poll every 30 seconds

print('Export task finished.')
